In [44]:
#pip install selenium

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time


import re


In [46]:

def return_mode(soup):
    soup_mode = str(soup.find(class_ = "tw"))
    if re.search(r'full/(.*)_text', soup_mode).group(1) == "s":
        return "Single"
    return "Double"

def return_song_name(soup):
    return soup.find(class_ = "song_name").get_text()

def return_score(soup):
    return soup.find(class_ = "txt_v").get_text()

def return_grade(soup):
    soup_grade = str(soup.find(class_ = "img"))
    return re.search(r'grade/(.*).png', soup_grade).group(1)

def return_plate(soup):
    soup_grade = str(soup.find(class_ = "img st1"))
    return re.search(r'plate/(.*).png', soup_grade).group(1)

def parse_soup_into_record_list(soup):
    if soup.find(class_ = "no_con") != None:
        return []
    soup_full = soup.find(class_ = "my_best_scoreList flex wrap")
    soup_list_raw = soup_full.find_all(class_ = "in")
    ans = []
    if len(soup_list_raw) == 0:
        return []
    for item in soup_list_raw:
        game_mode = return_mode(item)
        song_name = return_song_name(item)
        score = return_score(item)
        grade = return_grade(item)
        plate = return_plate(item)
        ans.append({"game_mode": game_mode, "song_name": song_name, "score": score, "grade": grade, "plate": plate})

    
    return ans

def crawl_record_by_level_unit(driver, level, page):
    driver.get(f'https://piugame.com/my_page/my_best_score.php?lv={level}&&page={page}')
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    record = parse_soup_into_record_list(soup)
    #print(soup.find(class_ = "xi last"))
    hasnext = soup.find(class_ = "xi last") != None
    return record, hasnext

def crawl_record_by_level(driver, level):
    get_next = True
    total_record = []
    page_index = 1
    while(get_next):
        print("level: " + str(level)+ ", current index:" + str(page_index))
        record, get_next = crawl_record_by_level_unit(driver, level, page_index)
        for item in record:
            total_record.append(item)
        time.sleep(5)
        page_index += 1

    return total_record

#returns: song name, score, grade, plate


# Set up the Selenium WebDriver
def selenium_login(ID, password):
    driver = webdriver.Chrome()
    # Navigate to the login page
    driver.get('https://piugame.com')

    # Find the username and password input fields and enter your credentials
    username = driver.find_element(By.NAME, 'mb_id')  # Adjust the name attribute if necessary
    password = driver.find_element(By.NAME, 'mb_password')  # Adjust the name attribute if necessary

    username.send_keys('chorjgml@naver.com')
    password.send_keys('helloworld1!')
    password.send_keys(Keys.RETURN)
    #
    time.sleep(5)
    return driver
    # Submit the login form
    #river.find_element(By.CLASS_NAME, 'btn st1').click()
    # Wait for the page to load (adjust the sleep time if necessary)




In [47]:

driver = selenium_login('chorjgml@naver.com', 'helloworld1!')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.203)
Stacktrace:
	GetHandleVerifier [0x00007FF60DEE1522+60802]
	(No symbol) [0x00007FF60DE5AC22]
	(No symbol) [0x00007FF60DD17CE4]
	(No symbol) [0x00007FF60DCEDFDF]
	(No symbol) [0x00007FF60DD91E57]
	(No symbol) [0x00007FF60DDA98D1]
	(No symbol) [0x00007FF60DD8A923]
	(No symbol) [0x00007FF60DD58FEC]
	(No symbol) [0x00007FF60DD59C21]
	GetHandleVerifier [0x00007FF60E1E41BD+3217949]
	GetHandleVerifier [0x00007FF60E226157+3488183]
	GetHandleVerifier [0x00007FF60E21F0DF+3459391]
	GetHandleVerifier [0x00007FF60DF9B8E6+823622]
	(No symbol) [0x00007FF60DE65FBF]
	(No symbol) [0x00007FF60DE60EE4]
	(No symbol) [0x00007FF60DE61072]
	(No symbol) [0x00007FF60DE518C4]
	BaseThreadInitThunk [0x00007FFEB138257D+29]
	RtlUserThreadStart [0x00007FFEB20CAA48+40]


In [ ]:
total_record_24 = crawl_record_by_level(driver, 24)
total_record_23 = crawl_record_by_level(driver, 23)
total_record_22 = crawl_record_by_level(driver, 22)
total_record_21 = crawl_record_by_level(driver, 21)
total_record_20 = crawl_record_by_level(driver, 20)



In [50]:
import json
abel_record = {"20": total_record_20, "21": total_record_21, "22": total_record_22, "23": total_record_23, "24": total_record_24, "25": total_record_25}
json.dump(abel_record, open("abel_record.json", "w"), ensure_ascii = False)